In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

2024-03-12 11:05:15.069452: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-12 11:05:15.160204: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 11:05:16.644126: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


implementation of the Variational Recurrent
Neural Network (VRNN) from https://arxiv.org/abs/1506.02216
using unimodal isotropic gaussian distributions for 
inference, prior, and generating models.

In [2]:
class vrnn(models.Model):
    def __init__(self, x_dim, h_dim, z_dim, n_layers, bias=False):
        super(VRNN, self).__init__()
        
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim
        self.n_layers = n_layers
        self.EPS = tf.constant(1e-6) # very small values, avoid numerical problem during iterations
        
        # 1. feature-extracting transformations (\phi function definition)
        # -----------------------------------------------------------------
        # \phi^x_\tau(x)
        self.phi_x = models.Sequential([
            layers.Dense(h_dim, activation='relu'),
            layers.Dense(h_dim, activation='relu')
        ])
        # \phi^z_\tau(z)
        self.phi_z = models.Sequential([
            layers.Dense(h_dim, activation='relu')
        ])
        
        # 2. encoder (Eq.9)
        # -----------------------------------------------------------------
        # \phi^{enc}_\tau()
        self.enc = models.Sequential([
            layers.Dense(h_dim, activation='relu'),
            layers.Dense(h_dim, activation='relu')
        ])
        # \mu_{z,t}
        self.enc_mean = layers.Dense(z_dim)
        # \sigma_{z,t}
        self.enc_std = models.Sequential([
            layers.Dense(z_dim),
            layers.Softplus()
        ])
         # SoftPlus is a smooth approximation to 
         # the ReLU function and can be used to constrain the output of a machine to always be positive.
        
        # 3. prior (Eq.5)
        # -----------------------------------------------------------------
        # \phi^{prior}_\tau(h)
        self.prior = models.Sequential([
            layers.Dense(h_dim, activation='relu')
        ])
        # \mu_{0,t}
        self.prior_mean = layers.Dense(z_dim)
        # \sigma_{0,t}: Vector where the i-th element is the std of the i-th dimension.
        self.prior_std = models.Sequential([
            layers.Dense(z_dim),
            layers.Softplus()
        ])
        
        # 4. decoder (Eq.6)
        # -----------------------------------------------------------------
        # \phi^{dec}_\tau()
        self.dec = models.Sequential([
            layers.Dense(h_dim, activation='relu'),
            layers.Dense(h_dim, activation='relu')
        ])
        # \sigma_{x,t}
        self.dec_std = models.Sequential([
            layers.Dense(x_dim),
            layers.Softplus()
        ])
        # \mu_{x,t}
        self.dec_mean = models.Sequential([
            layers.Dense(x_dim, activation='sigmoid')
        ])
        
        # 5. recurrence (Eq.7)
        # -----------------------------------------------------------------
        # num_layers代表lstm层数。在TensorFlow中实现多层LSTM，你需要通过堆叠多个LSTM层来实现。
        # TensorFlow会自动为偏置项提供一个合适的初始值
        self.rnn = layers.LSTM(self.h_dim, return_sequences=True, return_state=True, 
                               recurrent_initializer='glorot_uniform')